In [1]:
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
import os
import random
import numpy as np

from keras_unet_collection import models

from tqdm import tqdm 

from PIL import Image
import cv2
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import KFold

2024-11-06 16:21:15.019305: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 16:21:15.813541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_classes = 1
k = 5


IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1

DATA_PATH = 'data/'

data_ids = next(os.walk(DATA_PATH+'/image'))[2]

X = np.zeros((len(data_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((len(data_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_ALLOCATOR']= 'cuda_malloc_async'


In [3]:
print('Resizing training images and masks')
for n, id_ in tqdm(enumerate(data_ids), total=len(data_ids)):   
    path = DATA_PATH
    img = imread(path + '/image/' + id_)[:,:]
    img = img.reshape(img.shape[0], img.shape[1], IMG_CHANNELS)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X[n] = img  #Fill empty X_train with values from img
    
    mask = imread(path + 'label/' + id_)
    mask = (mask >= 250)
    mask = np.expand_dims(resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant',  
                                      preserve_range=True), axis=-1)
    y[n] = mask 
    #plt.axis("off")
    #imshow(y[n])
    #plt.show()

Resizing training images and masks


100%|█████████████████████████████████████████████████████████████████████████████████| 838/838 [00:18<00:00, 46.21it/s]


In [ ]:

acc = []
jacc = []
f1 = []
prec = []
rec = []

kf = KFold(n_splits=k, shuffle=True, random_state=28)
f = 1

for train_ind, test_ind in kf.split(X):
    X_train, X_val, y_train, y_val = X[train_ind], X[test_ind], y[train_ind], y[test_ind]
    
    model = models.unet_plus_2d((512, 512, 1), [16, 32, 64, 128], n_labels=1,
                            stack_num_down=2, stack_num_up=2,
                            activation='LeakyReLU', output_activation='Sigmoid',
                            batch_norm=False, pool='max', unpool=False, deep_supervision=True, name='xnet')

    # Define metrics for each output
    metrics = ['accuracy', tf.keras.metrics.Recall(name='recall')]
    metrics_dict = {output_name: metrics for output_name in model.output_names}

    model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Use a single loss function for the final output
    metrics= metrics_dict
     )
    
    checkpoint_filepath = 'UnetPLUS_test' + str(f)+'fold.keras'
    callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_loss'),
            tf.keras.callbacks.TensorBoard(log_dir='logs'),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=False,
                monitor='val_recall',
                mode='max',
                save_best_only=True,
                verbose=1)]
    
    model.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=16, epochs=300, callbacks=callbacks)
    #model.save('Unet++Modelo1.keras')
    
    loss = model.history.history['loss']
    val_loss = model.history.history['val_loss']
    
    plt.figure()
    plt.plot( loss, 'r', label='Training loss')
    plt.plot( val_loss, 'bo', label='Validation loss')
    plt.title('Training and Validation Loss - Fold' + str(f))
    plt.xlabel('Epoch')
    plt.ylabel('Loss Value')
    plt.ylim([0, 1])
    plt.legend()
    plt.show()
    
    for i in range(0, len(X_val)):
        sample_image = X_val[i]
        sample_mask = y_val[i].astype(np.uint8).flatten()
        prediction = model.predict(sample_image[tf.newaxis, ...],verbose=0)[0]
        predicted_mask = (prediction > 0.5).astype(np.uint8).flatten()
            
        acc.append(accuracy_score(sample_mask, predicted_mask))
        jacc.append(jaccard_score(sample_mask, predicted_mask))
        f1.append(f1_score(sample_mask, predicted_mask))
        prec.append(precision_score(sample_mask, predicted_mask))
        rec.append(recall_score(sample_mask, predicted_mask))

    del model 

    tf.keras.backend.clear_session()

    f+=1
    
print("Accuracy: "+ str(np.mean(acc)) + "+- " + str(np.std(acc)))
print("Jaccard: "+ str(np.mean(jacc)) + "+- " + str(np.std(jacc)))
print("Dice: "+ str(np.mean(f1)) + "+- " + str(np.std(f1)))
print("Precision: "+ str(np.mean(prec)) + "+- " + str(np.std(prec)))
print("Recall: "+ str(np.mean(rec)) + "+- " + str(np.std(rec)))

----------
deep_supervision = True
names of output tensors are listed as follows ("sup0" is the shallowest supervision layer;
"final" is the final output layer):

	xnet_output_sup0_activation
	xnet_output_sup1_activation
	xnet_output_sup2_activation
	xnet_output_final_activation
Epoch 1/300


I0000 00:00:1730921458.424882   75313 service.cc:145] XLA service 0x7f379800dac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730921458.424915   75313 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-11-06 16:30:58.463641: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-06 16:31:00.837870: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


In [ ]:
print("Accuracy: "+ str(np.mean(acc)) + "+- " + str(np.std(acc)))
print("Jaccard: "+ str(np.mean(jacc)) + "+- " + str(np.std(jacc)))
print("Dice: "+ str(np.mean(f1)) + "+- " + str(np.std(f1)))
print("Precision: "+ str(np.mean(prec)) + "+- " + str(np.std(prec)))
print("Recall: "+ str(np.mean(rec)) + "+- " + str(np.std(rec)))

In [ ]:
acc_mean_fold = []
jacc_mean_fold = []
f1_mean_fold = []
prec_mean_fold = []
rec_mean_fold = []

acc_std_fold = []
jacc_std_fold = []
f1_std_fold = []
prec_std_fold = []
rec_std_fold = []


f = 0
kf = KFold(n_splits=k, shuffle=True, random_state=28)
for train_ind, test_ind in tqdm(kf.split(X), total=kf.get_n_splits(), desc="k-fold"):
    X_train, X_val, y_train, y_val = X[train_ind], X[test_ind], y[train_ind], y[test_ind]

    acc= []
    jacc = []
    f1 = []
    prec = []
    rec = []
    
    model = tf.keras.models.load_model('model_'+str(f+1)+'fold.h5')

    for i in range(0, len(X_val)):
        sample_image = X_val[i]
        sample_mask = y_val[i].astype(np.uint8).flatten()
        prediction = model.predict(sample_image[tf.newaxis, ...],verbose=0)[0]
        predicted_mask = (prediction > 0.5).astype(np.uint8).flatten()
            
        acc.append(accuracy_score(sample_mask, predicted_mask))
        jacc.append(jaccard_score(sample_mask, predicted_mask))
        f1.append(f1_score(sample_mask, predicted_mask))
        prec.append(precision_score(sample_mask, predicted_mask))
        rec.append(recall_score(sample_mask, predicted_mask))

    acc_mean_fold.append(np.mean(acc))
    acc_std_fold.append(np.std(acc))
    jacc_mean_fold.append(np.mean(jacc))
    jacc_std_fold.append(np.std(jacc))
    f1_mean_fold.append(np.mean(f1))
    f1_std_fold.append(np.std(f1))
    prec_mean_fold.append(np.mean(prec))
    prec_std_fold.append(np.std(prec))
    rec_mean_fold.append(np.mean(rec))
    rec_std_fold.append(np.std(rec))
    
    print("Model - Fold" + str(f+1)+ "Accuracy: " + str(acc_mean_fold[-1]))
    print("Model - Fold" + str(f+1)+ "Jaccard: " + str(jacc_mean_fold[-1]))
    print("Model - Fold" + str(f+1)+ "Dice: " + str(f1_mean_fold[-1]))
    print("Model - Fold" + str(f+1)+ "Precision: " + str(prec_mean_fold[-1]))
    print("Model - Fold" + str(f+1)+ "Recall: " + str(rec_mean_fold[-1]))

    f += 1

In [ ]:
print("Accuracy: "+ str(np.mean(acc_mean_fold)*100) + " +- " + str(np.std(acc_std_fold)*100))
print("Jaccard: "+ str(np.mean(jacc_mean_fold)*100) + " +- " + str(np.std(jacc_std_fold)*100))
print("Dice: "+ str(np.mean(f1_mean_fold)*100) + " +- " + str(np.std(f1_std_fold)*100))
print("Precision: "+ str(np.mean(prec_mean_fold)*100) + " +- " + str(np.std(prec_std_fold)*100))
print("Recall: "+ str(np.mean(rec_mean_fold)*100) + " +- " + str(np.std(rec_std_fold)*100))

In [ ]:
best_model = tf.keras.models.load_model('UNET3Plus 5-fold models/model_1fold.keras')

In [ ]:

kf = KFold(n_splits=k, shuffle=True, random_state=28)
for i, (train_ind, test_ind) in enumerate(kf.split(X)):
    if i == 0:
        X_train, X_val, y_train, y_val = X[train_ind], X[test_ind], y[train_ind], y[test_ind]
        break

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input image', 'True mask', 'Predicted mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]), cmap='gray')
    plt.axis('off')
  plt.show()
  
i = random.randint(0, len(X_val))
sample_image = X_val[i]
sample_mask = y_val[i]
prediction = best_model.predict(sample_image[tf.newaxis, ...])[0]
predicted_mask = (prediction > 0.5).astype(np.uint8)
display([sample_image, sample_mask,predicted_mask])

sample_mask = y_val[i].astype(np.uint8).flatten()
predicted_mask = predicted_mask.flatten()


print("Accuracy: "+ str(accuracy_score(sample_mask, predicted_mask)))
print("Jaccard: "+ str(jaccard_score(sample_mask, predicted_mask)))
print("Dice: "+ str((2*jaccard_score(sample_mask, predicted_mask))/(1+jaccard_score(sample_mask, predicted_mask))))
print("Precision: "+ str(precision_score(sample_mask, predicted_mask)))
print("Recall: "+ str(recall_score(sample_mask, predicted_mask)))

In [ ]:
predicted_mask.shape